In this week, you are required to implement a toy GATConv and SAGEConv based on document. Also, you need to implement both in PyG and DGL. In this work, you will get a further understanding of tensor-centric in PyG and graph-centric in DGL.

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import dgl
import dgl.function as fn
import torch.nn.functional as F
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class PyG_GATConv(MessagePassing):
  def __init__(self, in_channel, out_channel):
    super().__init__(aggr='add')

    self.W = nn.Linear(in_channel, out_channel, bias=False)
    self.a = nn.Linear(2*out_channel, 1, bias=False)
    self.dropout = nn.Dropout(0.1)
    
    nn.init.xavier_uniform_(self.W.weight, gain=1.414)
    nn.init.xavier_uniform_(self.a.weight, gain=1.414)

  def forward(self, x, edge_index):
    x = self.W(x)
    edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
    out = self.propagate(edge_index, x=x)
    return out

  def message(self, x_i, x_j, edge_index):
    x_cat = torch.cat([x_i, x_j], dim=-1)
    e = F.leaky_relu(self.a(x_cat), negative_slope=0.2)

    node_i = edge_index[1]
    e = self.dropout(e)
    e = torch.exp(e)
    input = torch.zeros_like(e)
    e_sum = torch.scatter_add(input=input, dim=0, index=node_i.unsqueeze(1), src=e)
    alpha = e / (e_sum[node_i] + 1e-16)

    return alpha * x_j

class PyG_SAGEConv(MessagePassing):
  def __init__(self, in_channel, out_channel):
    pass

  def forward(x, edge_index):
    pass

  def message(x, edge_index):
    pass

In [18]:
class DGL_GATConv(nn.Module):
  def __init__(self, in_channel, out_channel):
    super().__init__()
    self.out_channel = out_channel
    self.W = nn.Linear(in_channel, out_channel, bias=False)
    self.a = nn.Linear(2*out_channel, 1, bias=False)
    self.dropout = nn.Dropout(0.1)
    
    nn.init.xavier_uniform_(self.W.weight, gain=1.414)
    nn.init.xavier_uniform_(self.a.weight, gain=1.414)

  def forward(self, g, x):
    h = self.W(x)
    g = dgl.add_self_loop(g)
    g.ndata['h'] = h

    def msg_func(edges):
      x_cat = torch.cat([edges.dst['h'], edges.src['h']], dim=-1)
      e = F.leaky_relu(self.a(x_cat), negative_slope=0.2)
      return {'e': self.dropout(torch.exp(e))}

    g.apply_edges(msg_func)

    g.update_all(
      fn.copy_e('e', 'm'),
      fn.sum('m', 'e_sum')
    )

    v = g.edges()[1]
    g.edata['alpha'] = g.edata['e'] / (g.ndata['e_sum'][v] + 1e-16)

    g.update_all(
      fn.u_mul_e('h', 'alpha', 'm'),
      fn.sum('m', 'h_out')
    )

    out = g.ndata.pop('h_out')
    return out

class DGL_SAGEConv(nn.Module):
  def __init__(self, in_channel, out_channel):
    pass

  def forward(self, g, h):
    pass

If you want to check your answer, you can run the following code.

In [24]:
edge_index = torch.tensor([[0,1,1,2,2,4],[2,0,2,3,4,3]])
x = torch.ones((5, 8))
conv = PyG_GATConv(8, 4)
output = conv(x, edge_index)
print(output)
# conv = PyG_SAGEConv(8, 4)
# output = conv(x, edge_index)
# print(output)

src = torch.tensor([0, 1, 1, 2, 2, 4])
dst = torch.tensor([2, 0, 2, 3, 4, 3])
h = torch.ones((5, 8))
g = dgl.graph((src, dst))
conv = DGL_GATConv(8, 4)
output = conv(g, h)
print(output)
# conv = DGL_SAGEConv(8, 4)
# output = conv(g, h)
# print(output)

tensor([[-1.7913, -0.2408, -1.7165, -0.0298],
        [-1.7913, -0.2408, -1.7165, -0.0298],
        [-1.7913, -0.2408, -1.7165, -0.0298],
        [-1.7913, -0.2408, -1.7165, -0.0298],
        [-1.7913, -0.2408, -1.7165, -0.0298]], grad_fn=<ScatterAddBackward0>)
tensor([[-2.7729,  1.9300,  1.6950, -0.3808],
        [-2.7729,  1.9300,  1.6950, -0.3808],
        [-2.7729,  1.9300,  1.6950, -0.3808],
        [-2.7729,  1.9300,  1.6950, -0.3808],
        [-2.7729,  1.9300,  1.6950, -0.3808]], grad_fn=<GSpMMBackward>)
